# Automatically Discover Missing Physics by Embedding Machine Learning into Differential Equations

https://docs.sciml.ai/Overview/dev/showcase/missing_physics/

In [ ]:
#DrWatson setup
using DrWatson
@quickactivate "diff_gleam"

In [ ]:
# SciML Tools
using OrdinaryDiffEq, ModelingToolkit, SciMLSensitivity#,DataDrivenDiffEq, DataDrivenSparse
using Optimization, OptimizationOptimJL #, OptimizationOptimisers

# Standard Libraries
using LinearAlgebra, Statistics

# External Libraries
using ComponentArrays, Lux, Zygote, Plots, StableRNGs
gr()

# Set a random seed for reproducible behaviour
rng = StableRNG(1111)

In [ ]:
# using Pkg
# Pkg.add("SciMLSensitivity")
# Pkg.add("Zygote")
# Pkg.add("StableRNGs")

## Problem Setup

Lotka-Volterra equations

## Generate training data
Using the full equations

In [ ]:
function lotka!(du,u,p,t) #general form for ODE solvers
    α, β, γ, δ = p
    du[1] = α * u[1] - β * u[2] * u[1]
    du[2] = γ * u[1] * u[2] - δ * u[2]
end

# Experimental parameters
tspan = (0.0, 5.0)
u0 = 5.0f0 * rand(rng, 2)

In [ ]:
p_ = [1.3, 0.9, 0.9, 1.8]
prob = ODEProblem(lotka!, u0, tspan, p_)
solution = solve(prob, Vern7(), abstol = 1e-12, reltol = 1e-12, saveat = 0.25)
X = Array(solution)
t = solution.t

x_bar = mean(X, dims = 2)
noise_magniutde = 5e-3
Xₙ = X .+ (noise_magniutde * x_bar) .* randn(rng, eltype(X), size(X)) #eltype(X) = float64

plot(solution, alpha = 0.75, color = :black, label = ["True Data" nothing])
scatter!(t, transpose(Xₙ), color = :red, label = ["Noisy Data" nothing])

https://docs.sciml.ai/DiffEqDocs/stable/solvers/ode_solve/: Vern7 =  Verner's “Most Efficient” 7/6 Runge-Kutta method. (lazy 7th order interpolant).

## Definition of the UDE

In [ ]:
rbf(x) = exp.(-(x .^ 2))

https://en.wikipedia.org/wiki/Radial_basis_function

Note (ChatGPT generated): 
In Julia, the const keyword is used to declare constants. Constants are variables whose values cannot be changed after they are assigned a value. const is often used to declare global constants that are known at compile time and should not be modified during the execution of the program. Here are some common use cases for const in Julia:

In [ ]:
const U = Lux.Chain(
    Lux.Dense(2, 5, rbf), Lux.Dense(5, 5, tanh), Lux.Dense(5, 5, rbf),
    Lux.Dense(5,2)
)
p_nn, st = Lux.setup(rng, U)
const _st = st

In [ ]:
methods(U)

In [ ]:
U([1,1], p_nn, _st)

Define a hybrid model

In [ ]:
function ude_dynamics!(du, u, p, t, p_true)
    u_hat = U(u, p, _st)[1]
    du[1] = p_true[1] * u[1] + u_hat[1] #p_true[1] = alpha
    du[2] = -p_true[4] * u[2] + u_hat[2] #p_true[4] = gamma
end 
#insert known parameters
nn_dynamics!(du, u, p, t) = ude_dynamics!(du, u, p, t, p_)
#Define the problem
prob_nn = ODEProblem(nn_dynamics!,Xₙ[:,1], tspan, p_nn)

KEY: the parameter vector of the ODE are the WEIGHTS of the NN!

`remake` for modifying an ODEProblem: https://docs.sciml.ai/DiffEqDocs/stable/basics/problem/#Modification-of-problem-types

In [ ]:
function predict(θ, X = Xₙ[:,1],T = t)
    _prob = remake(prob_nn, u0 = X, tspan = (T[1], T[end]), p = θ)
    Array(solve(
        _prob, Vern7(), saveat = T, 
        abstol = 1e-6, reltol = 1e-6,
        sensealg = QuadratureAdjoint(autojacvec = ReverseDiffVJP(true))
    ))
end

https://docs.sciml.ai/SciMLSensitivity/stable/manual/differential_equation_sensitivities/#Choosing-a-Sensitivity-Algorithm on chose of calculating derivatives!

Comparison to the `diffeqflux.ipynb` notebook: there we do NOT make this prediction function ourselves, instead we use the built-in forward mode prediction of the `NeuralODE` interface?

In [ ]:
function loss(θ)
    X_hat = predict(θ)
    sum(abs2, Xₙ .- X_hat)
end

In [ ]:
Xₙ

In [ ]:
predict(p_nn)

In [ ]:
Xₙ .- predict(p_nn)

In [ ]:
abs2.(Xₙ .- predict(p_nn))

In [ ]:
plot(t, transpose(predict(p_nn)))

In [ ]:
loss(p_nn)

## Training
Lots of parameters -> use reverse mode AD with Zygote
Use https://docs.sciml.ai/Optimization/stable/API/optimization_function/ from Optim.jl.
Idea here is that optimisation function is of form:
$$
\min_u f(u,p)
$$
with generally speaking $u$ the state variables and $p$ the parameters. So in the case of classic optimisation like the Rosenbrock equation (https://en.wikipedia.org/wiki/Rosenbrock_function) $u$ is $x,y$ while $p$ are parameters of the function, so the goal is to find the optimum values of $x$ and $y$.
For our case, we want to find the optimal values of the NN weights, so we should treat these weights as the states ($u$) of our optimisation function!

In [ ]:
adtype = Optimization.AutoZygote()
optf = Optimization.OptimizationFunction((x,p) -> loss(x), adtype) 
optprob = Optimization.OptimizationProblem(optf, ComponentVector{Float64}(p_nn))

In [ ]:
println(typeof(p_nn))
println(typeof(ComponentVector{Float64}(p_nn)))

ComponentVector part of the `ComponentArrays.jl` package (https://jonniedie.github.io/ComponentArrays.jl/stable/quickstart/)
Optimisation strategy:
- First use ADAM for finiding a general area of the parameter space
- Then use BFGS for honing in on local minimum!

In [ ]:
#create callback function
losses = Float64[] #initialie an emtpy array!

callback = function(p, loss)
    push!(losses, loss)
    if length(losses) % 50 == 0
        println("Current loss after $(length(losses)) iterations: $(losses[end])")
    end
    return false #obliged to specify that optimization should NOT be halted!
end

using OptimizationFlux
ADAM()

In [ ]:
res1 = Optimization.solve(optprob, ADAM(), callback = callback, maxiters = 5000)
println("Training loss after $(length(losses)) iterations: $(losses[end])")

In [ ]:
optprob2 = Optimization.OptimizationProblem(optf, res1.u) #res1.u is the new u0!
using OptimizationOptimJL
res2 = Optimization.solve(optprob2, LBFGS(), callback = callback, maxiters = 3000)
println("Final training loss after $(length(losses)) iterations: $(losses[end])")

# Rename the best candidate
p_trained = res2.u

## Visualising the training

In [ ]:
pl_losses = plot(1:5000, losses[1:5000], yaxis = :log10, xaxis = :log10, label = "ADAM",
xlabel = "Iterations", ylabel = "Loss")
plot!(5001:length(losses), losses[5001:end], label = "BFGS")

In [ ]:
ts = first(solution.t):mean(diff(solution.t))/2:last(solution.t)
X_hat = predict(p_trained, Xₙ[:,1], ts)

plot_hybrtid = plot(ts, transpose(X_hat), color = [:blue :green], label = ["UDE Approximation u1" "UDE Approximation u2"])
scatter!(solution.t, transpose(Xₙ) , color = [:blue :green], label = ["Noisy measurements u1" "Noisy measurements u2"])

Very nice fit now :)

Compare the learned interactions with what was fitted

In [ ]:
# Ideal unknown interactions of the predictor
Ȳ = [-p_[2] * (X_hat[1, :] .* X_hat[2, :])'; p_[3] * (X_hat[1, :] .* X_hat[2, :])']
# Neural network guess
Ŷ = U(X_hat, p_trained, st)[1]

plot(ts, transpose(Ȳ), color = [:blue :green], label = ["real dynamics u1" "real dynamics u2"])
plot!(ts, transpose(Ŷ), color = [:blue :green], linestyle = :dashdot, label = ["estimated dynamics u1" "estimated dynamics u2"])

## Symbolic regression via sparse regression (SINDy based)